In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --q datasets

In [8]:
!pip install -q unsloth xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 32.6 MB/s eta 0:00:00


In [9]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = userdata.get('HF_TOKEN_READ'), # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.9: Fast Qwen2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Data Processing

In [17]:
import pandas as pd
from datasets import Dataset, DatasetDict
import re

csv_file_path = '/content/drive/MyDrive/augnito.csv'
df = pd.read_csv(csv_file_path)
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\ï\`\√\d\\n]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower().strip()
    batch["label"] = re.sub(chars_to_ignore_regex, '', batch["label"]).lower().strip()
    return batch

dataset = Dataset.from_pandas(df)

train_dataset = dataset.filter(lambda x: x['subdirectory'] == 'train')
test_dataset = dataset.filter(lambda x: x['subdirectory'] == 'test')
validate_dataset = dataset.filter(lambda x: x['subdirectory'] == 'validate')

data = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validate": validate_dataset
})

for split in data:
    data[split] = data[split].rename_column("phrase", "sentence")
    data[split] = data[split].rename_column("prompt", "label")
    data[split] = data[split].remove_columns([
        "subdirectory", "file_path", "audio_clipping", "audio_clipping:confidence",
        "background_noise_audible", "background_noise_audible:confidence",
        "overall_quality_of_the_audio", "quiet_speaker", "quiet_speaker:confidence"
    ])

    data[split] = data[split].map(remove_special_characters)

data


Filter:   0%|          | 0/6661 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6661 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6661 [00:00<?, ? examples/s]

Map:   0%|          | 0/381 [00:00<?, ? examples/s]

Map:   0%|          | 0/5895 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 381
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 5895
    })
    validate: Dataset({
        features: ['sentence', 'label'],
        num_rows: 385
    })
})

In [22]:
from datasets import concatenate_datasets

merged_dataset = concatenate_datasets([data['train'], data['test']])

merged_dataset = merged_dataset.rename_column("sentence", "instruction")
merged_dataset = merged_dataset.rename_column("label", "output")

merged_dataset = merged_dataset.map(lambda example: {
    "instruction": example["instruction"],
    "input": "",
    "output": example["output"],
    "text": "Given a sentence generated via a Speech to Text model, clean the sentence grammaticaly and make it sound natural. Then classify the speaker's medical conditon in the given sentence."
})


merged_dataset


Map:   0%|          | 0/6276 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 6276
})

In [23]:
alpaca_prompt = """Given a sentence generated via a Speech to Text model, clean the sentence grammaticaly and make it sound natural. Then classify the speaker's medical conditon in the given sentence.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Replace with your tokenizer's EOS token (e.g., "<|endoftext|>")

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Add the formatted prompt and EOS token
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

formatted_dataset = merged_dataset.map(formatting_prompts_func, batched=True)

train_df = formatted_dataset.to_pandas()
output_csv_path = '/content/LLM_train.csv'  # Replace with your desired file path
train_df.to_csv(output_csv_path, index=False)

print(f"Formatted dataset (train) with EOS tokens saved at: {output_csv_path}")


Map:   0%|          | 0/6276 [00:00<?, ? examples/s]

Formatted dataset (train) with EOS tokens saved at: /content/LLM_train.csv


In [24]:
from datasets import concatenate_datasets

merged_dataset2 = concatenate_datasets([data['validate']])

merged_dataset2 = merged_dataset2.rename_column("sentence", "instruction")
merged_dataset2 = merged_dataset2.rename_column("label", "output")

merged_dataset2 = merged_dataset2.map(lambda example: {
    "instruction": example["instruction"],
    "input": "",
    "output": example["output"],
    "text": "Given a sentence generated via a Speech to Text model, clean the sentence grammaticaly and make it sound natural. Then classify the speaker's medical conditon in the given sentence."
})


merged_dataset2

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 385
})

In [25]:
alpaca_prompt = """Given a sentence generated via a Speech to Text model, clean the sentence grammaticaly and make it sound natural. Then classify the speaker's medical conditon in the given sentence.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Replace with your tokenizer's EOS token (e.g., "<|endoftext|>")

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Add the formatted prompt and EOS token
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

formatted_dataset = merged_dataset2.map(formatting_prompts_func, batched=True)

val_df = formatted_dataset.to_pandas()
output_csv_path = '/content/LLM_val.csv'  # Replace with your desired file path
val_df.to_csv(output_csv_path, index=False)

print(f"Formatted dataset (val) with EOS tokens saved at: {output_csv_path}")


Map:   0%|          | 0/385 [00:00<?, ? examples/s]

Formatted dataset (val) with EOS tokens saved at: /content/LLM_val.csv


In [30]:
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

# Finetuning

In [28]:
# We now add LoRA adapters so we only need to update 1 to 10% of all parameters!
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2024.11.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 20,

        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/6276 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [42]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,276 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,0.182000
2,0.168800
3,0.165600
4,0.231400
5,0.182500
6,0.211100
7,0.229000
8,0.206700
9,0.176400
10,0.228100


In [43]:
model.push_to_hub("Kabir259/QWEN2-Medical", token = userdata.get('HF_TOKEN_READ'))
tokenizer.push_to_hub("Kabir259/QWEN2-Medical", token = userdata.get('HF_TOKEN_READ'))

README.md:   0%|          | 0.00/571 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Kabir259/QWEN2-Medical


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]